In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf 

2025-03-09 19:18:01.578158: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 19:18:01.639143: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 19:18:02.691800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
###Data Prep 
##1.curl -L -o ~/Workspace/dp/A.I./ML/DL/tf/Models/toxic-tweets-dataset.zip  https://www.kaggle.com/api/v1/datasets/download/ashwiniyer176/toxic-tweets-dataset
##2.unzip toxic-tweets-dataset.zip && ls *.csv

data_df = pd.read_csv("FinalBalancedDataset.csv")
data_df = data_df.drop(columns=["Unnamed: 0"])
print(data_df.head(1))


   Toxicity                                              tweet
0         0   @user when a father is dysfunctional and is s...


In [3]:
data_df["Toxicity"].value_counts()

Toxicity
0    32592
1    24153
Name: count, dtype: int64

In [4]:
for _ in range(10):
    shuffle_data = np.random.randint(len(data_df))
    random_sample = data_df.iloc[shuffle_data]
    #print(random_sample["Toxicity"], random_sample["tweet"])

In [5]:
#PRE-PROCESSING
#TextToVector 

In [6]:
from transformers import BertTokenizer
tag = data_df["Toxicity"]
tag = np.array(tag)
tweet = data_df["tweet"]
tweet = np.array(tweet)
sentences = tweet.tolist()
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenized_output = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
print(tag.shape)
print(tweet.shape)
#print(tokenized_output)

/home/lab/miniconda3/envs/cancer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(56745,)
(56745,)


In [7]:
# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((tokenized_output, tag))

# Apply transformations
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)

In [8]:
#pipeline
train = dataset.take(int(len(dataset)* .7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.3))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense

# Define input layers
input_ids = Input(shape=(484,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(484,), dtype=tf.int32, name="attention_mask")  # Optional

# Embedding layer
embedding = Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=32)(input_ids)

# Bidirectional LSTM layer
bi_lstm = Bidirectional(LSTM(32, activation='tanh'))(embedding)

# Fully connected layers
fc_1 = Dense(128, activation='relu')(bi_lstm)
fc_2 = Dense(256, activation='relu')(fc_1)
fc_3 = Dense(128, activation='relu')(fc_2)

# Output layer
output = Dense(1, activation='sigmoid')(fc_3)

# Define model with multiple inputs
model = Model(inputs=[input_ids, attention_mask], outputs=output, name="text-classifier")

# Compile the model
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# Print model summary
model.summary()
print(train.take(1)) 
# Function to map datasets to the correct format for model input
def map_dataset_to_input_and_labels(dataset):
    return dataset.map(lambda x, y: ({
        'input_ids': x['input_ids'],  # Access based on keys
        'attention_mask': x['attention_mask']
    }, y))


# Map datasets to the correct format
train_dataset = map_dataset_to_input_and_labels(train)
val_dataset = map_dataset_to_input_and_labels(val)

# Training the model with correctly formatted datasets
history = model.fit(
    train_dataset,
    epochs=1,
    batch_size=16,
    validation_data=val_dataset
)


Model: "text-classifier"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 484)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 484, 32)              927872    ['input_ids[0][0]']           
                                                                                                  
 bidirectional (Bidirection  (None, 64)                   16640     ['embedding[0][0]']           
 al)                                                                                              
                                                                                                  
 dense (Dense)               (None, 128)                  8320      ['bidirectional[

KeyError: in user code:

    File "/tmp/ipykernel_80697/2997865353.py", line 34, in None  *
        lambda x, y: ({'input_ids': x[0], 'attention_mask': x[1]}, y)
    File "/home/lab/miniconda3/envs/cancer/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 276, in __getitem__
        raise KeyError(

    KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'
